# Wide Residual network using tensorflow

paper: https://arxiv.org/pdf/1605.07146.pdf
________________________________
> Classification of Cifar 10 dataset.

> **The model that will be attempted is wide-dropout :**

<img src='w-resnet.png'>

In [2]:
# load the necessary
import matplotlib.pylab as plt
%matplotlib inline
import os
import numpy as np
import math as m
import sys
print('Python version:',sys.version)
import tensorflow as tf
print('TF version:',tf.__version__)
from data_utils import input_pipeline
slim = tf.contrib.slim
from tensorflow.contrib.layers.python.layers import initializers
from colorama import Fore, Style
from datetime import datetime
import time
import glob

Python version: 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
TF version: 1.8.0


## Model definition

In [2]:
# Define the conv function
KEEP_PROB = 1.
WEIGHT_DECAY = 0.0005
def resnet_block(input_layer, out_features, training=True, scope=None):
    
    with tf.variable_scope(scope):
        
        with slim.arg_scope([slim.batch_norm],
                            is_training = training, scale=True,
                            decay=0.9, epsilon=1e-5,updates_collections=None,
                            param_initializers={
                                "beta": tf.constant_initializer(value=0),
                                "gamma": tf.random_normal_initializer(mean=1., stddev=0.01)}):

            # 1st Conv 3x3

            # batch norm        
            net = slim.batch_norm(input_layer)
            # Relu
            net = tf.nn.relu(net)
            #convolution
            net = slim.conv2d(net, out_features)
            #dropout
            net = slim.dropout(net,keep_prob=KEEP_PROB,is_training=training)

            # 2nd Conv 3x3
            # batch norm
            net = slim.batch_norm(net)
            # Relu
            net = tf.nn.relu(net)
            #convolution
            net = slim.conv2d(net, out_features)

            #concat with the input                    
            net += input_layer
    
    return net

def wide_resnet(inputs,
                num_classes=10,
                is_training=True,
                k = 6,
                depth = 22,
                scope='wide_resnet'):
    
    # number of filters at various stages
    filters = [16, 16*k, 32*k, 64*k]
    # from their code https://github.com/szagoruyko/wide-residual-networks/blob/master/models/wide-resnet.lua
    assert (depth - 4) % 6 == 0, 'depth should be 6n+4'
    n = int((depth - 4) / 6)
    
    # Set the weights to a small value initially and check whether the output is ln(num_of_classes) = ln(10) ~2.3
    #weights_initializer = tf.random_uniform_initializer(0,0.001)
    
    # He et all uniform weights
    #weights_initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0, 
    #                                       mode='FAN_IN', uniform=True)):
            
    with tf.variable_scope(scope, reuse=None):
        with tf.contrib.framework.arg_scope([slim.conv2d], 
                                             normalizer_fn=None,
                                             activation_fn=None,
                                             padding='SAME',
                                             kernel_size=(3,3),
                                             weights_initializer=initializers.variance_scaling_initializer(),
                                             weights_regularizer=slim.l2_regularizer(WEIGHT_DECAY),
                                             biases_initializer=None):


            #one conv at the beginning (spatial size: 32x32)
            net = slim.conv2d(inputs, filters[0], stride=(1,1), scope='first_conv' )
            # batch norm        
            net = slim.batch_norm(net, is_training =is_training, scale=True, updates_collections=None)
            # Relu
            net = tf.nn.relu(net)

            #block 1 -- Stage 1 (spatial size: 32x32)
            net = slim.conv2d(net,filters[1], stride=1)
            net = slim.repeat(net,n-1, resnet_block, filters[1], training=is_training, scope='block_1')
            print('block1:',net.get_shape().as_list())

            #block 2 -- Stage 2 (spatial size: 16x16)
            net = slim.conv2d(net,filters[2], stride=2)
            net = slim.repeat(net,n-1, resnet_block, filters[2], training=is_training, scope='block_2')
            print('block2:',net.get_shape().as_list())

            #block 3 -- Stage 3 (spatial size: 8x8)
            net = slim.conv2d(net,filters[3], stride=2)
            net = slim.repeat(net,n-1, resnet_block, filters[3], training=is_training, scope='block_3')
            print('block3:',net.get_shape().as_list())

            #average pooling 8x8
            net = slim.avg_pool2d(net, kernel_size=[8,8],padding='SAME',scope='avg_pool')

            # batch norm        
            net = slim.batch_norm(net, is_training =is_training, scale=True, updates_collections=None)
            # Relu
            net = tf.nn.relu(net)
            
            #flatten
            net = slim.flatten(net, scope='flatten')

            #logits
            net = slim.fully_connected(net, num_classes, activation_fn=None, scope='logits', 
                                       biases_regularizer=slim.l2_regularizer(WEIGHT_DECAY))

            return net


In [3]:
#checking the above graph
tf.reset_default_graph()
net = wide_resnet(tf.placeholder(tf.float32,(None, 32, 32, 3)),num_classes=10, is_training=True)
print('Variables in graph:')
for var in tf.global_variables():
    print(var.name)

block1: [None, 32, 32, 96]
block2: [None, 16, 16, 192]
block3: [None, 8, 8, 384]
Variables in graph:
wide_resnet/first_conv/weights:0
wide_resnet/BatchNorm/beta:0
wide_resnet/BatchNorm/gamma:0
wide_resnet/BatchNorm/moving_mean:0
wide_resnet/BatchNorm/moving_variance:0
wide_resnet/Conv/weights:0
wide_resnet/block_1/block_1_1/BatchNorm/beta:0
wide_resnet/block_1/block_1_1/BatchNorm/gamma:0
wide_resnet/block_1/block_1_1/BatchNorm/moving_mean:0
wide_resnet/block_1/block_1_1/BatchNorm/moving_variance:0
wide_resnet/block_1/block_1_1/Conv/weights:0
wide_resnet/block_1/block_1_1/BatchNorm_1/beta:0
wide_resnet/block_1/block_1_1/BatchNorm_1/gamma:0
wide_resnet/block_1/block_1_1/BatchNorm_1/moving_mean:0
wide_resnet/block_1/block_1_1/BatchNorm_1/moving_variance:0
wide_resnet/block_1/block_1_1/Conv_1/weights:0
wide_resnet/block_1/block_1_2/BatchNorm/beta:0
wide_resnet/block_1/block_1_2/BatchNorm/gamma:0
wide_resnet/block_1/block_1_2/BatchNorm/moving_mean:0
wide_resnet/block_1/block_1_2/BatchNorm/m

## Create the graph and run it in a session
> The input pipelines for both train and validation can be dynamically switched. 

In [4]:
#Create the graph
N_EPOCH = 200
NUM_CLASSES = 10
BATCH_SIZE = 128
LEARNING_RATE = 0.1 #3e-4 
WIDTH = 32
HEIGHT = 32
LOG_FREQ = 250
N_TRAIN_SAMPLES = 50000
CIFAR_LOCAL_FOLDER = '/home/vijay/datasets/image/cifar-10/cifar-10-batches-py'
MODEL_DIR = './slim_model/'

tf.reset_default_graph()

train_graph = tf.Graph()
with train_graph.as_default():
    global_step = tf.train.get_or_create_global_step()
    
    #-------------------------------------------
    #1. Create valid and train iterators
    #-------------------------------------------

    # placeholder for passing testing dataset
    test_filenames = tf.placeholder(tf.string, shape=[None], name='input')
    train_filenames = sorted(glob.glob(os.path.join(CIFAR_LOCAL_FOLDER, '%s*' %'train*.tfrecords')))
    
    # Create valid and train iterators
    train_dataset = input_pipeline(train_filenames, BATCH_SIZE)
    test_dataset = input_pipeline(test_filenames, BATCH_SIZE*3,  validation=True)
    
    # A feedable iterator is defined by a handle placeholder and its structure.
    handle = tf.placeholder(tf.string, shape=[], name='handle')

    iterator = tf.data.Iterator.from_string_handle(handle, 
                                                   train_dataset.output_types, 
                                                   train_dataset.output_shapes)
    
    # Returns a batch of image and mask at every call
    image_batch, label_batch = iterator.get_next()
    
    image_batch = tf.identity(image_batch, 'images')
    label_batch = tf.identity(label_batch, 'labels')
    
    # Create a initializable iterator for valid dataset, 
    # so that the dataset is same for every valid loop.
    testing_iterator = test_dataset.make_initializable_iterator()
    training_iterator = train_dataset.make_initializable_iterator() #make_one_shot_iterator()
    
    
    # EDIT : the below not working for inference -https://github.com/tensorflow/tensorflow/issues/11679
    # Build the iterator SaveableObject.
    #saveable = tf.contrib.data.make_saveable_from_iterator(testing_iterator)

    # Save the iterator state by adding it to the saveable objects collection.
    #tf.add_to_collection(tf.GraphKeys.SAVEABLE_OBJECTS, saveable)
    
    #------------------------------------------------
    # 2. load Wide-resnet to the graph
    #------------------------------------------------
    
    # Placeholder definitions to handle incase of testing individual images
    X = tf.placeholder_with_default(image_batch, shape=[None,HEIGHT,WIDTH,3], name='X')
    y = tf.placeholder_with_default(label_batch, shape=[None], name='y')
    
    # A bool to switch between training loop and testing loop
    is_train = tf.placeholder(dtype=tf.bool, name='is_train')
    
    #wide-resnet logits
    logits = wide_resnet(X, num_classes=10, is_training=is_train)
    
    #------------------------------------------------
    # 3. Loss and accuracy
    #------------------------------------------------
        
    loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(y,NUM_CLASSES), logits=logits)
    
    #add regularization loss
    regularization_loss = tf.add_n(tf.losses.get_regularization_losses())
    loss += regularization_loss
        
    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    
    # use Momentum optimizer    
    solver = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True)

    # create the train op
    train_op = slim.learning.create_train_op(loss, solver, global_step=global_step)
    
    # Probabilities of the outputs
    prob = tf.nn.softmax(logits, name='prob')
    
    # Prediction 
    pred = tf.argmax(prob, 1, name='pred')

    #accuracy create two different ops
    with tf.name_scope('train_metrics'):
        train_acc_op, train_accuracy = tf.metrics.accuracy(labels=tf.cast(y, tf.int32), predictions=tf.cast(pred, tf.int32))
    
    with tf.name_scope('test_metrics'):
        test_acc_op, test_accuracy = tf.metrics.accuracy(labels=tf.cast(y, tf.int32), predictions=tf.cast(pred, tf.int32))
    
    # Select accuracy for saving
    accuracy = tf.cond(is_train, lambda:train_accuracy, lambda:test_accuracy)
    accuracy = tf.identity(accuracy,name='accuracy')
    
    # Create summaries
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)    
    tf.summary.scalar('learning_rate', learning_rate)
    my_summary_op = tf.summary.merge_all()


block1: [None, 32, 32, 96]
block2: [None, 16, 16, 192]
block3: [None, 8, 8, 384]


In [5]:
# Run the graph in the session

def testing_loop(_handle, file_names, writer, test=False):
    """ Testing/validation loop between trainings"""
    # do validation at each epoch
    _loss = 0.
    count = 0
    # Reset the iterator at every valid call, to go through the validation set
    sess.run(testing_iterator.initializer, {test_filenames: file_names})

    while True:
        try:
            _summaries, _l, _acc, _,_pred, _label = sess.run([my_summary_op, loss,test_accuracy, test_acc_op, pred, y], 
                                               {learning_rate:LEARNING_RATE, is_train:False, handle:_handle})  
            count += len(_label)
            _loss += _l*len(_label)
        except tf.errors.OutOfRangeError:
            break

    _loss = _loss/count
    
    if test:
        tf.logging.info(Fore.CYAN + Style.BRIGHT +'Step:{}: Test Loss:{:.5f}: Test Accuracy:{:.5f}'  
                .format(step,_loss, _acc)+Fore.RESET+Style.RESET_ALL)  
    else:
        tf.logging.info(Fore.GREEN + Style.BRIGHT +'Step:{}: Valid Loss:{:.5f}:Valid Accuracy:{:.5f}'  
                .format(step,_loss, _acc)+Fore.RESET+Style.RESET_ALL)  

    #write validation summaries
    writer.add_summary(_summaries, step)
    return _loss

with tf.Session(graph=train_graph) as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    valid_files = sorted(glob.glob(os.path.join(CIFAR_LOCAL_FOLDER, '%s*' %'valid*.tfrecords')))
    test_files = sorted(glob.glob(os.path.join(CIFAR_LOCAL_FOLDER, '%s*' %'eval*.tfrecords')))
    train_handle = sess.run(training_iterator.string_handle())
    valid_handle = sess.run(testing_iterator.string_handle(), {test_filenames: valid_files})
    test_handle = sess.run(testing_iterator.string_handle(), {test_filenames: test_files})
    
    # variables for early exit
    best_v_loss = 10e10
    early_stopping = 0
    saver = tf.train.Saver()
    start_time = time.time()

    # Get the test/valid metrics variables to reset at each call
    stream_vars_test = [v for v in tf.local_variables() if 'test_metrics/' in v.name]
    # Get the train metrics variables to reset at each epoch
    stream_vars_train = [v for v in tf.local_variables() if 'train_metrics/' in v.name]

    #Summaries for validation and training
    summaries_dir = os.path.join(MODEL_DIR,'summaries')
    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sess.graph)
    train_writer.flush()
    valid_writer = tf.summary.FileWriter(summaries_dir + '/valid', sess.graph)
    valid_writer.flush()
    test_writer = tf.summary.FileWriter(summaries_dir + '/test', sess.graph)
    test_writer.flush()

    lr = LEARNING_RATE    
    for e in range(N_EPOCH):
        
        # Learning rate changes
        if e > 60 and e <= 120:
            lr = 0.02
        elif e > 120 and e <= 160:
            lr = 0.004
        elif e > 160 :
            lr = 0.0008
        
        tf.logging.info(Fore.RED+ Style.BRIGHT +'Epoch:{}'.format(e)+Fore.RESET+Style.RESET_ALL)
        # initialize the training iterator at every epoch
        sess.run(training_iterator.initializer)              
        while True:

            try:
                # The train images are passed
                _, l, _, step = sess.run([train_op, loss, train_acc_op, global_step], {learning_rate:lr, is_train:True,handle:train_handle})  

                #print train logs
                if (step %LOG_FREQ) == 0:
                    train_summaries,acc, l, reg_loss = sess.run([my_summary_op, train_accuracy, loss,regularization_loss], 
                                                                {learning_rate:lr,is_train:True,handle:train_handle})  
                    #Print train logs
                    tf.logging.info(Fore.BLUE + Style.BRIGHT +'Step:{} Time:{:.5f} Reg Loss:{:.5f} Train Loss:{:.5f}: Accuracy:{:.5f}' 
                                    .format(step,time.time()-start_time,reg_loss,l,acc)+Fore.RESET+Style.RESET_ALL)
                    #train_pred, train_label = sess.run([pred, y], {is_train:True, handle:train_handle} )
                    start_time = time.time()

                    # write train summaries
                    train_writer.add_summary(train_summaries, step)


            except tf.errors.OutOfRangeError:
                break
        
        # do validation at each epoch
        valid_loss = testing_loop(valid_handle, valid_files, valid_writer)
                
        # reset train and validation statistics every epoch
        sess.run(tf.variables_initializer(stream_vars_train))
        sess.run(tf.variables_initializer(stream_vars_test))

        # Save the best model based on valid loss and test the model on the testing set
        if (best_v_loss > valid_loss) and (step > 0):
            tf.logging.info(Fore.RED+ Style.BRIGHT +'Saving the model...'+Fore.RESET+Style.RESET_ALL)
            saver.save(sess, os.path.join(MODEL_DIR, 'wide_resnet'))
            best_v_loss = valid_loss
            early_stopping = 0
            
            # test when the validation set has a new minimum loss 
            
            testing_loop(test_handle, test_files, test_writer, test=True)
       
            #reset test statistics
            sess.run(tf.variables_initializer(stream_vars_test))

        
        else:
            early_stopping += 1


        #if early_stopping > 3:
        #    tf.logging.info(Fore.RED+ Style.BRIGHT +'Stopping the training...'+Fore.RESET+Style.RESET_ALL)
        #    break
    #One final testing    
    testing_loop(test_handle, test_files, test_writer, test=True)


INFO:tensorflow:Epoch:0
INFO:tensorflow:Step:0 Time:5.24136 Reg Loss:1.69910 Train Loss:21.68912: Accuracy:0.11719
INFO:tensorflow:Step:250 Time:22.45802 Reg Loss:1.91426 Train Loss:3.90737: Accuracy:0.19531
INFO:tensorflow:Step:500 Time:22.56979 Reg Loss:1.49333 Train Loss:3.42177: Accuracy:0.22396
INFO:tensorflow:Step:621: Valid Loss:3.29116:Valid Accuracy:0.23315
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:621: Test Loss:3.28660: Test Accuracy:0.23870
INFO:tensorflow:Epoch:1
INFO:tensorflow:Step:750 Time:28.87526 Reg Loss:1.16951 Train Loss:2.98130: Accuracy:0.23904
INFO:tensorflow:Step:1000 Time:22.55122 Reg Loss:0.91625 Train Loss:2.72598: Accuracy:0.23957
INFO:tensorflow:Step:1244: Valid Loss:2.66959:Valid Accuracy:0.25907
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:1244: Test Loss:2.67048: Test Accuracy:0.25880
INFO:tensorflow:Epoch:2
INFO:tensorflow:Step:1250 Time:28.32624 Reg Loss:0.71814 Train Loss:2.47673: Accuracy:0.25908
INFO:tensorflow:Step:1500 

INFO:tensorflow:Step:9750 Time:22.71223 Reg Loss:0.22710 Train Loss:1.21663: Accuracy:0.63641
INFO:tensorflow:Step:9959: Valid Loss:1.48144:Valid Accuracy:0.54368
INFO:tensorflow:Epoch:16
INFO:tensorflow:Step:10000 Time:26.32023 Reg Loss:0.23060 Train Loss:0.71753: Accuracy:0.85156
INFO:tensorflow:Step:10250 Time:22.69041 Reg Loss:0.23531 Train Loss:1.21822: Accuracy:0.76562
INFO:tensorflow:Step:10500 Time:22.69992 Reg Loss:0.23948 Train Loss:1.18226: Accuracy:0.74479
INFO:tensorflow:Step:10581: Valid Loss:1.10738:Valid Accuracy:0.70830
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:10581: Test Loss:1.12680: Test Accuracy:0.70250
INFO:tensorflow:Epoch:17
INFO:tensorflow:Step:10750 Time:28.50953 Reg Loss:0.23915 Train Loss:0.79890: Accuracy:0.70389
INFO:tensorflow:Step:11000 Time:22.72378 Reg Loss:0.24511 Train Loss:1.36133: Accuracy:0.70252
INFO:tensorflow:Step:11204: Valid Loss:1.29438:Valid Accuracy:0.62646
INFO:tensorflow:Epoch:18
INFO:tensorflow:Step:11250 Time:26.33625 R

INFO:tensorflow:Step:21500 Time:22.74432 Reg Loss:0.35000 Train Loss:1.06686: Accuracy:0.80469
INFO:tensorflow:Step:21750 Time:22.71922 Reg Loss:0.35384 Train Loss:1.18051: Accuracy:0.75521
INFO:tensorflow:Step:21786: Valid Loss:1.11423:Valid Accuracy:0.74942
INFO:tensorflow:Epoch:35
INFO:tensorflow:Step:22000 Time:26.28107 Reg Loss:0.34904 Train Loss:0.76847: Accuracy:0.83594
INFO:tensorflow:Step:22250 Time:22.70350 Reg Loss:0.35605 Train Loss:1.22529: Accuracy:0.77734
INFO:tensorflow:Step:22409: Valid Loss:1.19803:Valid Accuracy:0.71246
INFO:tensorflow:Epoch:36
INFO:tensorflow:Step:22500 Time:26.35152 Reg Loss:0.35634 Train Loss:0.67222: Accuracy:0.88281
INFO:tensorflow:Step:22750 Time:22.69727 Reg Loss:0.35719 Train Loss:1.08759: Accuracy:0.81641
INFO:tensorflow:Step:23000 Time:22.74250 Reg Loss:0.36049 Train Loss:1.08551: Accuracy:0.79948
INFO:tensorflow:Step:23031: Valid Loss:1.03198:Valid Accuracy:0.76531
INFO:tensorflow:Epoch:37
INFO:tensorflow:Step:23250 Time:26.32641 Reg Loss:

INFO:tensorflow:Epoch:54
INFO:tensorflow:Step:33750 Time:26.46167 Reg Loss:0.40165 Train Loss:0.75608: Accuracy:0.88281
INFO:tensorflow:Step:34000 Time:22.67755 Reg Loss:0.40588 Train Loss:1.01059: Accuracy:0.83594
INFO:tensorflow:Step:34237: Valid Loss:1.05836:Valid Accuracy:0.78003
INFO:tensorflow:Epoch:55
INFO:tensorflow:Step:34250 Time:26.26764 Reg Loss:0.40572 Train Loss:0.66813: Accuracy:0.92188
INFO:tensorflow:Step:34500 Time:22.73114 Reg Loss:0.39978 Train Loss:1.19488: Accuracy:0.81641
INFO:tensorflow:Step:34750 Time:23.21973 Reg Loss:0.40736 Train Loss:1.14501: Accuracy:0.78125
INFO:tensorflow:Step:34859: Valid Loss:1.05078:Valid Accuracy:0.78072
INFO:tensorflow:Epoch:56
INFO:tensorflow:Step:35000 Time:27.07753 Reg Loss:0.39948 Train Loss:0.67714: Accuracy:0.90625
INFO:tensorflow:Step:35250 Time:23.77614 Reg Loss:0.40600 Train Loss:0.96595: Accuracy:0.85547
INFO:tensorflow:Step:35482: Valid Loss:0.96379:Valid Accuracy:0.81230
INFO:tensorflow:Epoch:57
INFO:tensorflow:Step:3550

INFO:tensorflow:Step:44500 Time:23.76137 Reg Loss:0.22859 Train Loss:0.51302: Accuracy:0.94922
INFO:tensorflow:Step:44750 Time:23.36665 Reg Loss:0.23100 Train Loss:0.78754: Accuracy:0.90104
INFO:tensorflow:Step:44819: Valid Loss:0.64525:Valid Accuracy:0.86633
INFO:tensorflow:Epoch:72
INFO:tensorflow:Step:45000 Time:27.72532 Reg Loss:0.22568 Train Loss:0.30928: Accuracy:0.96094
INFO:tensorflow:Step:45250 Time:23.63725 Reg Loss:0.22746 Train Loss:0.60322: Accuracy:0.91797
INFO:tensorflow:Step:45442: Valid Loss:0.55399:Valid Accuracy:0.88846
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:45442: Test Loss:0.58278: Test Accuracy:0.88470
INFO:tensorflow:Epoch:73
INFO:tensorflow:Step:45500 Time:29.35872 Reg Loss:0.22751 Train Loss:0.25687: Accuracy:0.88606
INFO:tensorflow:Step:45750 Time:22.92435 Reg Loss:0.22355 Train Loss:0.50912: Accuracy:0.88621
INFO:tensorflow:Step:46000 Time:23.19592 Reg Loss:0.22685 Train Loss:0.56226: Accuracy:0.88598
INFO:tensorflow:Step:46064: Valid Loss:0

...

INFO:tensorflow:Step:69098: Valid Loss:0.60087:Valid Accuracy:0.88665
INFO:tensorflow:Epoch:111
INFO:tensorflow:Step:69250 Time:28.89129 Reg Loss:0.24508 Train Loss:0.26134: Accuracy:1.00000
INFO:tensorflow:Step:69500 Time:24.87845 Reg Loss:0.24718 Train Loss:0.62880: Accuracy:0.93359
INFO:tensorflow:Step:69721: Valid Loss:0.57125:Valid Accuracy:0.89762
INFO:tensorflow:Epoch:112
INFO:tensorflow:Step:69750 Time:26.83391 Reg Loss:0.25048 Train Loss:0.29439: Accuracy:0.99219
INFO:tensorflow:Step:70000 Time:23.92849 Reg Loss:0.24425 Train Loss:0.52969: Accuracy:0.94141
INFO:tensorflow:Step:70250 Time:23.17836 Reg Loss:0.24910 Train Loss:0.54753: Accuracy:0.92448
INFO:tensorflow:Step:70343: Valid Loss:0.63617:Valid Accuracy:0.88290
INFO:tensorflow:Epoch:113
INFO:tensorflow:Step:70500 Time:27.15786 Reg Loss:0.24557 Train Loss:0.26225: Accuracy:1.00000
INFO:tensorflow:Step:70750 Time:23.78353 Reg Loss:0.24716 Train Loss:0.55485: Accuracy:0.95703
INFO:tensorflow:Step:70966: Valid Loss:0.55485:

INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:80303: Test Loss:0.46051: Test Accuracy:0.92860
INFO:tensorflow:Epoch:129
INFO:tensorflow:Step:80500 Time:28.53016 Reg Loss:0.21397 Train Loss:0.28815: Accuracy:0.92911
INFO:tensorflow:Step:80750 Time:22.70179 Reg Loss:0.21236 Train Loss:0.29605: Accuracy:0.92970
INFO:tensorflow:Step:80926: Valid Loss:0.45211:Valid Accuracy:0.93038
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:80926: Test Loss:0.46149: Test Accuracy:0.92830
INFO:tensorflow:Epoch:130
INFO:tensorflow:Step:81000 Time:28.47709 Reg Loss:0.21086 Train Loss:0.21743: Accuracy:0.92921
INFO:tensorflow:Step:81250 Time:22.71725 Reg Loss:0.20895 Train Loss:0.25511: Accuracy:0.92989
INFO:tensorflow:Step:81500 Time:22.73870 Reg Loss:0.20768 Train Loss:0.37342: Accuracy:0.93037
INFO:tensorflow:Step:81548: Valid Loss:0.45740:Valid Accuracy:0.93191
INFO:tensorflow:Epoch:131
INFO:tensorflow:Step:81750 Time:26.34100 Reg Loss:0.20584 Train Loss:0.22313: Accuracy:0.99219
IN

INFO:tensorflow:Step:91500 Time:22.73108 Reg Loss:0.16057 Train Loss:0.22484: Accuracy:0.98177
INFO:tensorflow:Step:91508: Valid Loss:0.44390:Valid Accuracy:0.92354
INFO:tensorflow:Epoch:147
INFO:tensorflow:Step:91750 Time:26.33907 Reg Loss:0.15919 Train Loss:0.27732: Accuracy:0.96094
INFO:tensorflow:Step:92000 Time:22.73981 Reg Loss:0.15870 Train Loss:0.36282: Accuracy:0.94922
INFO:tensorflow:Step:92131: Valid Loss:0.43463:Valid Accuracy:0.92629
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:92131: Test Loss:0.44647: Test Accuracy:0.92390
INFO:tensorflow:Epoch:148
INFO:tensorflow:Step:92250 Time:28.51518 Reg Loss:0.15791 Train Loss:0.15950: Accuracy:0.92486
INFO:tensorflow:Step:92500 Time:22.68846 Reg Loss:0.15687 Train Loss:0.36272: Accuracy:0.92473
INFO:tensorflow:Step:92750 Time:22.71060 Reg Loss:0.15673 Train Loss:0.28825: Accuracy:0.92527
INFO:tensorflow:Step:92753: Valid Loss:0.45488:Valid Accuracy:0.92142
INFO:tensorflow:Epoch:149
INFO:tensorflow:Step:93000 Time:26.34

INFO:tensorflow:Step:102500 Time:22.69705 Reg Loss:0.13752 Train Loss:0.20429: Accuracy:0.93106
INFO:tensorflow:Step:102714: Valid Loss:0.37793:Valid Accuracy:0.93652
INFO:tensorflow:Epoch:165
INFO:tensorflow:Step:102750 Time:26.35270 Reg Loss:0.13728 Train Loss:0.14225: Accuracy:1.00000
INFO:tensorflow:Step:103000 Time:22.72455 Reg Loss:0.13701 Train Loss:0.18378: Accuracy:0.99609
INFO:tensorflow:Step:103250 Time:22.73265 Reg Loss:0.13678 Train Loss:0.24242: Accuracy:0.98177
INFO:tensorflow:Step:103336: Valid Loss:0.37343:Valid Accuracy:0.93846
INFO:tensorflow:Saving the model...
INFO:tensorflow:Step:103336: Test Loss:0.41116: Test Accuracy:0.93220
INFO:tensorflow:Epoch:166
INFO:tensorflow:Step:103500 Time:28.50628 Reg Loss:0.13652 Train Loss:0.14393: Accuracy:0.93296
INFO:tensorflow:Step:103750 Time:23.26422 Reg Loss:0.13627 Train Loss:0.24112: Accuracy:0.93331
INFO:tensorflow:Step:103959: Valid Loss:0.37462:Valid Accuracy:0.93535
INFO:tensorflow:Epoch:167
INFO:tensorflow:Step:104000

INFO:tensorflow:Step:114541: Valid Loss:0.38276:Valid Accuracy:0.93654
INFO:tensorflow:Epoch:184
INFO:tensorflow:Step:114750 Time:26.31119 Reg Loss:0.12593 Train Loss:0.14960: Accuracy:0.99219
INFO:tensorflow:Step:115000 Time:23.25227 Reg Loss:0.12572 Train Loss:0.19091: Accuracy:0.99219
INFO:tensorflow:Step:115164: Valid Loss:0.38121:Valid Accuracy:0.93701
INFO:tensorflow:Epoch:185
INFO:tensorflow:Step:115250 Time:27.68497 Reg Loss:0.12550 Train Loss:0.12586: Accuracy:1.00000
INFO:tensorflow:Step:115500 Time:23.54693 Reg Loss:0.12526 Train Loss:0.17470: Accuracy:0.98828
INFO:tensorflow:Step:115750 Time:23.12553 Reg Loss:0.12506 Train Loss:0.14728: Accuracy:0.98698
INFO:tensorflow:Step:115786: Valid Loss:0.38575:Valid Accuracy:0.93625
INFO:tensorflow:Epoch:186
INFO:tensorflow:Step:116000 Time:26.84012 Reg Loss:0.12483 Train Loss:0.15179: Accuracy:0.97656
INFO:tensorflow:Step:116250 Time:23.07332 Reg Loss:0.12461 Train Loss:0.15285: Accuracy:0.98047
INFO:tensorflow:Step:116409: Valid Lo

The best model was saved at *epoch 165* and the corresponding **Test Accuracy: 0.93220**

> INFO:tensorflow:Epoch:165  
INFO:tensorflow:Step:102750 Time:26.35270 Reg Loss:0.13728 Train Loss:0.14225: Accuracy:1.00000  
INFO:tensorflow:Step:103000 Time:22.72455 Reg Loss:0.13701 Train Loss:0.18378: Accuracy:0.99609  
INFO:tensorflow:Step:103250 Time:22.73265 Reg Loss:0.13678 Train Loss:0.24242: Accuracy:0.98177  
INFO:tensorflow:Step:103336: Valid Loss:0.37343:Valid Accuracy:0.93846  
INFO:tensorflow:Saving the model...  
INFO:tensorflow:Step:103336: Test Loss:0.41116: Test Accuracy:0.93220  
    